# Save and Resume a Keras MNIST ConvNet Model

This jupyter notebook, show you how to save and resume a Keras Model. In this example we will use the Deep Learning hello-world!: the MNIST classification task.

Note: to run code cell you have to press **`Shift + Enter`**.

### Import Packages

First we need a single point with all the dependencies:

In [1]:
from __future__ import print_function
import keras
import os.path
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


### Hyper Parameters and Variables

Even for Hyper-Parameters and Variables is a good practice have a single point, this improve code readability and experiments interation.

In [10]:
# Path to saved model weights(as hdf5)
resume_weights = "/model/mnist-cnn-best.hdf5"

# Where to save Checkpoint(In the /output folder)
filepath = "/output/mnist-cnn-best.hdf5"

# Hyper-parameters
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

### Data Processing and Transformation

Next, we process the dataset sample in tensor, ready to be feed into the model.

In [3]:
# MNIST handwritten image classification
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape strategy according to backend
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    # 1 x 28 x 28 [number_of_channels (colors) x height x weight]
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    # 28 x 28 x 1 [height x weight x number_of_channels (colors)]
    input_shape = (img_rows, img_cols, 1)

# Reshape, type, normalized, print
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Dataset info
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

10682368/11490434 [==========================>...] - ETA: 0sx_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


### Define the Model

A ConvNet Model, state of the art for image classification task.

In [4]:
# MODEL
# Conv(32,3,3)[ReLU] -> Conv(64,3,3)[ReLU] -> MaxPool(2,2)[Dropout 0.25] ->
# FC(_, 128)[ReLU][Dropout 0.5] -> FC(128, 10)[Softmax]
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                    activation='relu',
                    input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

### Resume a checkpoint

Run the following line if you want to resume an existing checkpoint.

In [11]:
# If exists a best model, load its weights!
if os.path.isfile(resume_weights):
    print ("Resumed model's weights from {}".format(resume_weights))
    # load weights
    model.load_weights(resume_weights)

Resumed model's weights from ./mnist-cnn-best.hdf5


### Define The Loss Function and The Optimizers

In this example we use the Cross Entropy Loss and Adam Optimizer.

In [6]:
# CEE, Adam
model.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=keras.optimizers.Adam(),
            metrics=['accuracy'])

### Checkpoint Strategy

The strategy we have adopted for the this example is the following:
- Keep only one checkpoints
- Trigger the strategy at the end of every epoch
- Save the one with the best(max) validation accuracy 

In [7]:
# Keep only a single checkpoint, the best over test accuracy.
checkpoint = ModelCheckpoint(filepath,
                            monitor='val_acc',
                            verbose=1,
                            save_best_only=True,
                            mode='max')

### Training

Let's train the model and see our checkpoint strategy in action.

In [12]:
# Train
model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                verbose=1,
                validation_data=(x_test, y_test),
                callbacks=[checkpoint])

# Eval
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 9s - loss: 0.0210 - acc: 0.9929 - val_loss: 0.0309 - val_acc: 0.9912
Epoch 2/12
60000/60000 [==============================] - 9s - loss: 0.0207 - acc: 0.9931 - val_loss: 0.0248 - val_acc: 0.9927
Epoch 3/12
60000/60000 [==============================] - 9s - loss: 0.0205 - acc: 0.9934 - val_loss: 0.0270 - val_acc: 0.9922
Epoch 4/12
60000/60000 [==============================] - 9s - loss: 0.0186 - acc: 0.9940 - val_loss: 0.0279 - val_acc: 0.9928
Epoch 5/12
60000/60000 [==============================] - 9s - loss: 0.0171 - acc: 0.9944 - val_loss: 0.0273 - val_acc: 0.9924
Epoch 6/12
60000/60000 [==============================] - 9s - loss: 0.0153 - acc: 0.9948 - val_loss: 0.0289 - val_acc: 0.9928
Epoch 7/12
60000/60000 [==============================] - 9s - loss: 0.0157 - acc: 0.9944 - val_loss: 0.0296 - val_acc: 0.9936
Epoch 8/12
60000/60000 [==============================] - 9s 

### Resume the checkpoint after the training

Let's take a look at the checkpoint just created. (you should see the `mnist-cnn-best.hdf5` file)

In [9]:
% ls

MNIST_data/     command.sh                     mnist-cnn-best.hdf5
README.md       keras_mnist_cnn.py             pytorch_mnist_cnn.py
Untitled.ipynb  keras_mnist_cnn_jupyter.ipynb  pytorch_mnist_cnn_jupyter.ipynb


Jupyter Notebook run in the `/output` folder, so it's here.
If you want to load it, go to the Hyper parameters and Varables Code Cell, replace the resume weigths var in this way:
`# Path to saved model weights(as hdf5)
resume_weights = "./mnist-cnn-best.hdf5"`, run the cell, go to the **Resume a checkpoint** Code Cell, run it, and rerun the **Training Code Cell**, that's it.